In [1]:
import cptac
import cptac.utils as ut
import pandas as pd

In [ ]:
ccrcc = cptac.Ccrcc()
en = cptac.Endometrial()
luad = cptac.Luad()
hnscc  = cptac.Hnscc()
lscc = cptac.Lscc()
pdac = cptac.Pdac()

In [ ]:
cancers = [ccrcc, en, luad, hnscc, lscc, pdac]
cancer_names = ['CCRCC', 'Endometrial', 'LUAD', 'HNSCC', 'LSCC', 'PDAC']

In [ ]:
cancer_dfs = []
for cancer, cancer_name in zip(cancers, cancer_names):
    prot_normal_df = cancer.get_proteomics('normal')
    if isinstance(prot_normal_df.columns, pd.MultiIndex):
        prot_normal_df = ut.reduce_multiindex(df= prot_normal_df, levels_to_drop = 'Database_ID')
    trans_normal_df = cancer.get_transcriptomics('normal')
    if isinstance(trans_normal_df.columns, pd.MultiIndex):
        trans_normal_df = ut.reduce_multiindex(df = trans_normal_df, levels_to_drop='Database_ID')
    prot_normal_df['Patient_ID'] = prot_normal_df.index
    trans_normal_df['Patient_ID'] = trans_normal_df.index
    prot_normal_df = prot_normal_df.melt(id_vars = 'Patient_ID', var_name = 'Gene', value_name = 'Proteomic')
    trans_normal_df = trans_normal_df.melt(id_vars = 'Patient_ID', var_name = 'Gene', value_name = 'Transcriptomics')
    prot_normal_df['Stage'] = ['Normal'] * len(prot_normal_df)
    trans_normal_df['Stage'] = ['Normal'] * len(trans_normal_df)
    if cancer == ccrcc:
        stage = cancer.get_clinical('tumor')[['tumor_stage_pathological']]
        stage['Patient_ID'] = stage.index
        stage = stage.rename(columns = {'tumor_stage_pathological': 'Stage'})
        stage = stage.reset_index(level = 0, drop = True)
    elif cancer == luad or cancer == lscc:
        stage = cancer.get_clinical('tumor')[['Stage']]
        stage['Patient_ID'] = stage.index
        stage = stage.reset_index(level = 0, drop = True)
        stages = []
        for s in stage['Stage']:
            if s in ['1A', '1B', '1', 1, 'I', 'IA', 'IB']:
                stages.append('Stage I')
            elif s in ['2A', '2B', 'II', 'IIA', 'IIB']:
                stages.append('Stage II')
            elif s in ['3A', '3', 3, 'III', 'IIIA', 'IIIB']:
                stages.append('Stage III')
            elif s in ['IV']:
                stages.append('Stage IV')
            else:
                stages.append(float('NaN'))
        stage['Stage'] = stages
    elif cancer == en:
        stage = cancer.get_clinical('tumor')[['FIGO_stage']]
        stage['Patient_ID'] = stage.index
        stage = stage.rename(columns = {'FIGO_stage': 'Stage'})
        stage = stage.reset_index(level = 0, drop = True)
        stages = []
        for s in stage['Stage']:
            if s in ['IA', 'IB']:
                stages.append('Stage I')
            elif s in ['II']:
                stages.append('Stage II')
            elif s in ['IIIA', 'IIIB', 'IIIC2', 'IIIC1']:
                stages.append('Stage III')
            elif s in ['IVB', 'IV']:
                stages.append('Stage IV')
            else:
                stages.append(float('NaN'))
        stage['Stage'] = stages
    elif cancer == hnscc:
        stage = cancer.get_clinical('tumor')[['patho_staging_curated']]
        stage['Patient_ID'] = stage.index
        stage = stage.rename(columns = {'patho_staging_curated': 'Stage'})
        stage = stage.reset_index(level = 0, drop = True)
        
    elif cancer == pdac:
        stage = cancer.get_clinical('tumor')[['tumor_stage_pathological']]
        stage['Patient_ID'] = stage.index
        stage = stage.rename(columns = {'tumor_stage_pathological': 'Stage'})
        stage = stage.reset_index(level = 0, drop = True)
        stages = []
        for s in stage['Stage']:
            if s in ['Stage IA', 'Stage IB']:
                stages.append('Stage I')
            elif s in ['Stage IIA', 'Stage IIB']:
                stages.append('Stage II')
            elif s in ['Stage III']:
                stages.append('Stage III')
            elif s in ['Stage IV']:
                stages.append('Stage IV')
            else:
                stages.append(float('NaN'))
        stage['Stage'] = stages
        
        
    prot_tumor_df = cancer.get_proteomics('tumor')
    if isinstance(prot_tumor_df.columns, pd.MultiIndex):
        prot_tumor_df = ut.reduce_multiindex(df= prot_tumor_df, levels_to_drop = 'Database_ID')
    trans_tumor_df = cancer.get_transcriptomics('tumor')
    if isinstance(trans_tumor_df.columns, pd.MultiIndex):
        trans_tumor_df = ut.reduce_multiindex(df = trans_tumor_df, levels_to_drop='Database_ID')
    prot_tumor_df['Patient_ID'] = prot_tumor_df.index
    trans_tumor_df['Patient_ID'] = trans_tumor_df.index
    prot_tumor_df = prot_tumor_df.melt(id_vars = 'Patient_ID', var_name = 'Gene', value_name = 'Proteomic')
    trans_tumor_df = trans_tumor_df.melt(id_vars = 'Patient_ID', var_name = 'Gene', value_name = 'Transcriptomics')
    trans_tumor_df = pd.merge(trans_tumor_df, stage, how = 'outer')
    prot_tumor_df = pd.merge(prot_tumor_df, stage, how = 'outer')
    prot_df = pd.merge(prot_tumor_df, prot_normal_df, how = 'outer')
    trans_df = pd.merge(trans_normal_df, trans_tumor_df, how = 'outer')
    cancer_df = pd.merge(trans_df, prot_df,how = 'outer')
    cancer_df['Cancer'] = [cancer_name] * len(cancer_df)
    cancer_dfs.append(cancer_df)
prot_trans_stages_df = pd.concat(cancer_dfs)
prot_trans_stages_df

In [ ]:
prot_trans_stages_df = prot_trans_stages_df.dropna()
prot_trans_stages_df

In [ ]:
df = prot_trans_stages_df
df = df.drop(columns = 'Patient_ID')
grouped_stages = df.groupby(['Cancer', 'Stage', 'Gene'])
corr_df = grouped_stages.apply(lambda x: x.corr(method = 'pearson', min_periods = 10))
corr_df = corr_df.drop(columns = 'Proteomic')
corr_df.reset_index(inplace = True)
corr_df = corr_df[corr_df.level_3 == 'Proteomic']
corr_df = corr_df.drop(columns = 'level_3')
corr_df = corr_df.rename(columns = {'Transcriptomics': 'PearsonR'})
corr_df = corr_df.dropna()
corr_df


In [ ]:
corr_df.to_csv('data/Cancer_stages_correlations.csv', index = False)